In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    height_shift_range = 0.1,
    width_shift_range = 0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    zoom_range=0.1
)

test_datagen = ImageDataGenerator(
    rescale=1./255.
)

# Load the train and test dataset
train_generator = train_datagen.flow_from_directory(
    'dataset/makanan/train',
    batch_size=32,
    target_size=(224, 224),
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'dataset/makanan/test',
    batch_size=32,
    target_size=(224, 224),
    class_mode='categorical'
)

Found 6478 images belonging to 23 classes.
Found 2100 images belonging to 23 classes.


In [2]:
import tensorflow_hub as hub
ResNet_V2_50 = 'https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5'
Inception_V3 = 'https://tfhub.dev/google/imagenet/inception_v3/classification/5'
MobileNet_V3_Large = 'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/classification/5'
Inception_ResNet_V2 = 'https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/5'
NASNet_Large = 'https://tfhub.dev/google/imagenet/nasnet_large/classification/5'


In [3]:
# use GPU
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
  except RuntimeError as e:
    print(e)

# model_ResNet = tf.keras.Sequential([
#     hub.KerasLayer(ResNet_V2_50, trainable = False, input_shape = (224,224,3), name = 'Resnet_V2_50'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1024, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(23, activation = 'softmax', name = 'Output_layer')
# ])

# model_Inception = tf.keras.Sequential([
#     hub.KerasLayer(Inception_V3, trainable = False, input_shape = (224,224,3), name = 'Inception_V3'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1024, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(23, activation = 'softmax', name = 'Output_layer')
# ])

# model_MobileNet = tf.keras.Sequential([
#     hub.KerasLayer(MobileNet_V3_Large, trainable = False, input_shape = (224,224,3), name = 'MobileNet_V3_Large'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1024, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(23, activation = 'softmax', name = 'Output_layer')
# ])

model_Inception_ResNet = tf.keras.Sequential([
    hub.KerasLayer(Inception_ResNet_V2, trainable = False, input_shape = (224,224,3), name = 'Inception_ResNet_V2'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(23, activation = 'softmax', name = 'Output_layer')
])

# model_NASNet = tf.keras.Sequential([
#     hub.KerasLayer(NASNet_Large, trainable = False, input_shape = (331,331,3), name = 'NASNet_Large'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1024, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(23, activation = 'softmax', name = 'Output_layer')
# ])

In [4]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') > 0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 30:
        lr *= 0.1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [5]:
model = model_Inception_ResNet

In [6]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

model_history = model.fit(
    train_generator,
    epochs=50,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    validation_steps=test_generator.n // test_generator.batch_size,
    validation_data=test_generator,
    callbacks=[callbacks, lr_scheduler, early_stopping]
)

Epoch 1/50
202/202 [==============================] - 99s 396ms/step - loss: 3.0060 - accuracy: 0.4576 - val_loss: 2.1782 - val_accuracy: 0.6279 - lr: 0.0010
Epoch 2/50
202/202 [==============================] - 74s 365ms/step - loss: 2.2660 - accuracy: 0.5661 - val_loss: 1.9389 - val_accuracy: 0.6298 - lr: 0.0010
Epoch 3/50
202/202 [==============================] - 74s 364ms/step - loss: 2.0092 - accuracy: 0.6022 - val_loss: 1.8175 - val_accuracy: 0.6519 - lr: 0.0010
Epoch 4/50
202/202 [==============================] - 73s 363ms/step - loss: 1.8327 - accuracy: 0.6298 - val_loss: 1.7896 - val_accuracy: 0.6317 - lr: 0.0010
Epoch 5/50
202/202 [==============================] - 73s 361ms/step - loss: 1.7754 - accuracy: 0.6379 - val_loss: 1.6737 - val_accuracy: 0.6529 - lr: 0.0010
Epoch 6/50
202/202 [==============================] - 71s 350ms/step - loss: 1.7285 - accuracy: 0.6319 - val_loss: 1.6166 - val_accuracy: 0.6760 - lr: 0.0010
Epoch 7/50
202/202 [==============================] 

KeyboardInterrupt: 

In [ ]:
import datetime
model.save('model_InceptionResnet_Makanan_{}.keras'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

In [ ]:

import tensorflow as tf
import tensorflow_hub as hub

model = tf.keras.models.load_model('model_InceptionResnet_Makanan_20210609-153911.keras',
                                   custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
# predict the test dataset again and show the result, every image with wrong prediction will be shown and ignored if the prediction is correct
import numpy as np
import matplotlib.pyplot as plt
import os

class_names = os.listdir('dataset/jajanan/test')

wrong_predictions = []
wrong_images = []
wrong_labels = []
wrong_pred_percentage = np.zeros(len(class_names))
wrong_actual_percentage = np.zeros(len(class_names))
total_class_data = np.zeros(len(class_names))

test_generator.reset()
for k in range(np.ceil(test_generator.n / test_generator.batch_size).astype(int)):
  test_images, test_labels = next(test_generator)
  predictions = model.predict(test_images)
  for i in range(len(predictions)):
    if class_names[np.argmax(predictions[i])] != class_names[np.argmax(test_labels[i])]:
      wrong_predictions.append(predictions[i])
      wrong_images.append(test_images[i])
      wrong_labels.append(test_labels[i])
      wrong_pred_percentage[np.argmax(predictions[i])] += 1
      wrong_actual_percentage[np.argmax(test_labels[i])] += 1
    total_class_data[np.argmax(test_labels[i])] += 1

print('Total wrong prediction: ', len(wrong_predictions))
print('Accuracy: ', (test_generator.n - len(wrong_predictions)) / test_generator.n)
print('Wrong prediction percentage: ')
for i in range(len(wrong_pred_percentage)):
  print(class_names[i], ': ', wrong_pred_percentage[i], '/', total_class_data[i])
print('='*25)
print('Wrong Missclasified percentage: ')
for i in range(len(wrong_actual_percentage)):
  print(class_names[i], ': ', wrong_actual_percentage[i], '/', total_class_data[i])

fig = plt.figure(figsize=(20, 60))
for i in range(len(wrong_predictions) // 5):
  for j in range(5):
    ax = fig.add_subplot(len(wrong_predictions) // 5, 5, i * 5 + j + 1)
    ax.imshow(wrong_images[i * 5 + j])
    ax.set_title('Prediction: ' + class_names[np.argmax(wrong_predictions[i * 5 + j])] + '\nActual: ' + class_names[np.argmax(wrong_labels[i * 5 + j])] + '\nFilename: ' + test_generator.filenames[i * 5 + j])
    ax.axis('off')
